## Concept detection
This notebook shows how to create a simple concept detection (https://arxiv.org/abs/1610.01644, https://arxiv.org/abs/1711.11279) pipeline using all the tools found in this repo.

The main point of this is to show an use-case where it would be nice to be able to do something beyond just basic inference (which is done better through the lc0-binary itself, by converting your model to the ONNX-format anyway)

In [4]:
# First, let's make our concept dataset. I included a big file of PGNs to make this a bit easier.
# (This is from the elite-database, https://database.nikonoel.fr/)
with open("database.pgn") as f:
    all_pgns = f.read().split("\n\n")

In [9]:
import chess

# For our concept-function, we define it to be whether the player to move is in check.

def concept_func(board: chess.Board):
    return board.is_check()

In [ ]:
%cd ..

In [13]:
import io
import chess.pgn
from leela_board import LeelaBoard

positive_samples = []
negative_samples = []
number_of_samples = 5000
# Now we can go through the PGNs one-by-one.
for pgn in all_pgns:
    # We read the pgn by using python-chess
    game = chess.pgn.read_game(io.StringIO(pgn))
    if game is not None:
        # Here, the main star of the show appears.
        data_board = LeelaBoard()
        for move in game.mainline_moves():
            # We push a move, updating the internal state...
            data_board.push(move)
            # And check if the player to move is in check (handy reference to the internal python-chess board)
            if concept_func(data_board.pc_board):
                # And get the current position in the correct input format!
                positive_samples.append(data_board.lcz_features())
            else:
                negative_samples.append(data_board.lcz_features())
    
    # Make sure that we don't keep a lot of these around
    positive_samples = positive_samples[:number_of_samples]
    negative_samples = negative_samples[:number_of_samples]
    # Stop when we have a balanced data set
    if len(positive_samples) >= number_of_samples and len(negative_samples) >= number_of_samples:
        break
    

In [14]:
import tensorflow.keras as keras

# Now, we can get our activations.
# First, let's load the network.
# (Make sure that you unzipped the archive provided by this repo first though.)
network = keras.models.load_model("models/tf_model")
# Then, we make a submodel from the input to the tenth block of the model.
# (You can look at the architecture just like a normal model, in this case by looking at model.summary())
network.trainable = False
# We attach a linear probe:
inter_flat = keras.layers.Flatten()(network.get_layer("block10/conv1").output)
probe_output = keras.layers.Dense(1, activation="sigmoid", kernel_regularizer=keras.regularizers.L1(0.01))(inter_flat)
probe_model = keras.Model(network.inputs, probe_output)
probe_model.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(), run_eagerly=True)

In [15]:
import numpy as np

# Finally, the concept detection.
# We make our training and testing dataset...
labels = np.array([0] * len(negative_samples) + [1] * len(positive_samples))
inputs = np.concatenate([negative_samples, positive_samples])

shuffle_indices = np.arange(len(labels))
np.random.shuffle(shuffle_indices)
labels = labels[shuffle_indices]
inputs = inputs[shuffle_indices]

x_train, x_test, y_train, y_test = inputs[:4000], inputs[4000:], labels[:4000], labels[4000:]

In [ ]:
probe_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=16)